# OpenAI Swarm: A Framework for Multi-Agent Orchestration 

In [ ]:
!pip install git+https://github.com/openai/swarm.git

In [14]:
import logging

# Configure logging to display in the notebook
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

# Create a stream handler to output logs to the notebook
stream_handler = logging.StreamHandler()
stream_handler.setLevel(logging.INFO)

# Add a simple format to the handler
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
stream_handler.setFormatter(formatter)

# Add the handler to the logger
logger.addHandler(stream_handler)

## Arcade-AI

In [15]:
from arcadepy import Arcade
import os
from dotenv import load_dotenv

load_dotenv()

arcade_client = Arcade()

In [16]:
# Wrapper to execute ArcadeAI tools
def Google_CreateEvent(*args, **kwargs):

    response = arcade_client.tools.execute(
        tool_name="Google_CreateEvent",
        inputs=kwargs,
        user_id=os.environ['ARCADE_USER_ID'],
    )
    print(response)
    logger.info(response)
    return str(response)

In [ ]:
from langchain_core.utils.function_calling import convert_to_openai_tool
from langchain_arcade._utilities import tool_definition_to_pydantic_model

def update_defaults(schema, defaults_list):
    """
    Update the default field of each key in the schema considering a list of dictionaries with keys to update.
    
    Args:
        schema (dict): The schema to update.
        defaults_list (list[dict]): A list of dictionaries with default values to update.
    """
    # Merge all dictionaries in the list into one
    merged_defaults = {}
    for default_dict in defaults_list:
        merged_defaults.update(default_dict)
    
    # Update the schema with the merged defaults
    for key, value in schema['function']['parameters']['properties'].items():
        if key in merged_defaults:
            value['default'] = merged_defaults[key]
    return schema

default_args = [
    {
        "attendee_emails": []
    }
]

def get_schema(tool_id, default_args):
    schema = convert_to_openai_tool(tool_definition_to_pydantic_model(arcade_client.tools.get(tool_id=tool_id)))
    schema = update_defaults(schema, default_args)
    schema['function']['name'] = tool_id
    logger.info(schema)
    return schema

schema = get_schema("Google_CreateEvent", default_args)

In [ ]:
schema

## Main Agent using Tools

In [19]:
from swarm_custom.core import Agent, Swarm
from typing import List, Callable, Dict

AGENT_SYSTEM_PROMPT = """
# Task
Tu nombre es Gabriela y sos un asistente de IA diseñado para ayudar al equipo de Pampa Labs. 
                      
# Guidelines                   
Tus respuestas deben ser:

1. Amigables y accesibles, usando un tono cálido
2. Concisas y al grano, evitando verbosidad innecesaria
3. Útiles e informativas, proporcionando información precisa
4. Respetuosas de la privacidad del usuario y los límites éticos

Solo puedes ayudar usando las herramientas disponibles y con pedidos que vengan de miembros del equipo. Todo lo que no se pueda responder usando las herramientas, debes decir que no puedes ayudar y disculparte.
"""

# I had to convert Pydantic models to function arguments with corresponding docstrings
# This involves extracting fields from the Pydantic models and creating function
# parameters with appropriate type hints and descriptions in the docstring.
def set_meal_tool(context_variables, meal: str, date: str):
    """
    Sets the meal plan for a specific date.

    Args:
        meal (str): The name of the meal.
        date (str): The date of the meal plan.
    """
    return f"Meal plan created and sent to the provider: {meal} for {date} by team member {context_variables['id']}"

def get_expenses_tool(context_variables):
    """
    Retrieves all pending expenses
    """

    # Mock implementation for testing purposes
    mock_expenses = {
        "user1": {
            'expenses': [
                {
                "expense_type": "food",
                "date": "2023-05-01",
                "total_value": 50.0,
                "state": "pending"
            },
            {
                "expense_type": "coffee",
                "date": "2023-05-02",
                "total_value": 5.0,
                "state": "pending"
                }
            ]
        }
    }
    # Functions called by an Agent should return a type that has __str__
    return f"Expenses retrieved: {mock_expenses[context_variables['id']]['expenses']}"

class GabyAgent:
    def __init__(self,
        prompt = AGENT_SYSTEM_PROMPT,
        model_name: str = "gpt-4o",
        tools: List = [],
    ):

        self.agent = Agent(
            name="Main Agent",
            instructions=prompt,
            functions=tools,
            model=model_name
        )
        self.client = Swarm()

    def invoke(self, id, messages, debug=False):
        response = self.client.run(
            agent=self.agent,
            messages=messages,
            debug=debug,
            context_variables={"id": id}
        )
        return response.messages[-1]["content"]
    
    def stream(self, id, messages, debug=False):
        stream = self.client.run(
            agent=self.agent,
            messages=messages,
            debug=debug,
            context_variables={"id": id},
            stream=True
        )
        for chunk in stream:
            yield chunk

In [ ]:
gaby_agent = GabyAgent(tools=[set_meal_tool, get_expenses_tool, {"function": Google_CreateEvent, "schema": schema}])
response_content = gaby_agent.invoke(
    id="user1",
    messages=[
        {"role": "user", "content": "I want to set the meal plan for the date 2024-05-01. The meal is lasagna."},
        {"role": "user", "content": "I want to get the expenses"},
        {"role": "user", "content": "Calendar event for the date 2024-10-31 at 10:00 PM - 11:00 PM argentina timezone. The event is a meeting with Halloween party."}
    ],
    debug=True
)
print(response_content)